In [1]:
import cv2
import imutils
import numpy as np
import glob

In [2]:
with np.load('calibration_values.npz') as calibration_values:
    mtx, dist, rvecs, tvecs = [calibration_values[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]

In [3]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [8]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*5, 1, 3), np.float32) 
objp[:,:,:2] = np.mgrid[0:7,  0:5].T.reshape(-1,1,2)
print(objp[5])
print(cv2.convertPointsToHomogeneous(objp[5]))

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

[[ 5.  0.  0.]]
[[[ 5.  0.  0.  1.]]]


In [84]:
def pixelToWorld(img_point, z, rvecs, tvecs, mtx, obj_to_robot_mtx):
    '''
    Prints world coordinate with given z value of inputted pixel coordinate.
    Input: img_point (1x2 numpy array u, v), Z value of world point, rvecs, tvecs, camera mtx (3x3)
    Returns 1x3 world point [[X, Y, Z]]
    '''
    if img_point.shape != (1, 2):
        raise ValueError('Expects 1x2 numpy array as img_point, received {}'.format(img_point))
    # 3x3 rotation matrix
    rtmtx, _ = cv2.Rodrigues(rvecs)
    # 3 x 4 rotation transformation matrix
    rt_tr_mtx = np.concatenate((rtmtx, tvecs), axis=1)
    # complete projection matrix
    transform_mtx = np.dot(mtx, rt_tr_mtx)
    u = img_point[0][0]
    v = img_point[0][1]
    # To solve for X,Y given Z, solves
    # desired_vec = desired_mat x [[X,Y]]
    desired_vec = np.array([[transform_mtx[0][2] * z + transform_mtx[0][3] - u * transform_mtx[2][2] * z - u * transform_mtx[2][3]],
                           [transform_mtx[1][2] * z + transform_mtx[1][3] - v * transform_mtx[2][2] * z - v * transform_mtx[2][3]]])
    
    desired_mat = np.array([[u * transform_mtx[2][0] - transform_mtx[0][0], u * transform_mtx[2][1] - transform_mtx[0][1]],
                           [v * transform_mtx[2][0] - transform_mtx[1][0], v * transform_mtx[2][1] - transform_mtx[1][1]]])
    inv_dmat = np.linalg.inv(desired_mat)
    pred = np.dot(inv_dmat, desired_vec)
    print(pred)
    pred_homogeneous = cv2.convertPointsToHomogeneous(pred.transpose())[0].transpose()
    pred_robot = np.dot(obj_to_robot_mtx, pred_homogeneous)
    print("pred_robot: {}".format(pred_robot))
    pred_robot[2] = z
#     z_axis = np.array([[z]])
#     print(pred.shape)
#     print(z_axis.shape)
#     pred = np.concatenate((pred, z_axis), axis=0)
    print("Prediction: {}".format(pred_robot))
    return pred_robot

In [85]:
def getLocations(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        img_points = np.array([[x, y]])
        pixelToWorld(img_points, 0, rvecs, tvecs, mtx, obj_to_robot_mtx)

In [86]:
def calibrateRobotTransformation(robot_points):
    '''
    TODO:
    handle inconsistent points, generalize if needed
    Input: object points (0,0), (6,0), (0,4) in robot_points (2x1x2)
    Output: (robot to object, object to robot) homogeneous transformation matrices
    '''
    t0 = robot_points[0][0]
    t1 = robot_points[0][1]
    r00 = (robot_points[1][0] - t0) / 6
    r01 = (robot_points[1][1]  - t1) / 6
    print(r01)
    r10 = (robot_points[2][0] - t0) / 4
    r11 = (robot_points[2][1] - t1) / 4
    obj_to_robot_rot_mtx = np.array([[r00, r01], [r10, r11]])
    robot_to_obj_rot_mtx = np.linalg.inv(obj_to_robot_rot_mtx)
    print(t0)
    print(t1)
    # tvec_obj_to_robot = np.array([[t0 * r00 + t1 * r01], [t0 * r10 + t1 * r11]])
    tvec_obj_to_robot = np.array([[t0], [t1]])
    tvec_robot_to_obj = np.array([[-t0 * robot_to_obj_rot_mtx[0][0] - t1 * robot_to_obj_rot_mtx[0][1]],
                                 [-t0 * robot_to_obj_rot_mtx[1][0] - t1 * robot_to_obj_rot_mtx[1][1]]])
    obj_to_robot_mtx = np.concatenate((np.concatenate((obj_to_robot_rot_mtx, tvec_obj_to_robot), axis=1), [[0, 0, 1]]), axis=0)    
    robot_to_obj_mtx = np.concatenate((np.concatenate((robot_to_obj_rot_mtx, tvec_robot_to_obj), axis=1), [[0, 0, 1]]), axis=0)
    return (robot_to_obj_mtx, obj_to_robot_mtx)

# robot_to_obj_mtx, obj_to_robot_mtx = calibrateRobotTransformation(((40, 400), (202, 400), (40,292.15)))
robot_to_obj_mtx, obj_to_robot_mtx = calibrateRobotTransformation(((40, 400), (202, 400), (40,292.15)))
print(robot_to_obj_mtx)
print(obj_to_robot_mtx)

# def rigidTransformPoint(point, transform_mtx):
#     '''
#     Transform 3d homogeneous point (shape (3x1)) with passed in rigid transform matrix
#     '''
#     return np.dot
    
print(np.dot(robot_to_obj_mtx, np.array([[202, 292.15, 1]]).transpose()))
print(np.dot(obj_to_robot_mtx, np.array([[3, 3, 1]]).transpose()))

0.0
40
400
[[  0.03703704   0.          -1.48148148]
 [  0.          -0.03708855  14.83541956]
 [  0.           0.           1.        ]]
[[  27.        0.       40.    ]
 [   0.      -26.9625  400.    ]
 [   0.        0.        1.    ]]
[[ 6.]
 [ 4.]
 [ 1.]]
[[ 121.    ]
 [ 319.1125]
 [   1.    ]]


In [ ]:
cv2.namedWindow('img')
cv2.setMouseCallback('img', getLocations)

#for fname in glob.glob('checkerboard_images/*.jpg'):
for fname in ['checkerboard_images/img21.jpg']:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7,5),None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
#         print(corners2)
#         print(objp)

        # Find the rotation and translation vectors.
        #_, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, None)
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        img = draw(img,corners2,imgpts)
        print(imgpts.shape)
        cv2.drawChessboardCorners(img, (7,5), corners2,ret)
        cv2.imshow('img',img)
        k = cv2.waitKey(0) & 0xff
        if k == 's':
            cv2.imwrite(fname[:6]+'.png', img)

cv2.destroyAllWindows()

(3, 1, 2)
[[ 1.07907982]
 [-0.09035879]]
pred_robot: [[  69.13515509]
 [ 402.43629891]
 [   1.        ]]
Prediction: [[  69.13515509]
 [ 402.43629891]
 [   0.        ]]
[[ 0.01956457]
 [-0.06255973]]
pred_robot: [[  40.52824338]
 [ 401.68676665]
 [   1.        ]]
Prediction: [[  40.52824338]
 [ 401.68676665]
 [   0.        ]]
[[ 0.01956457]
 [-0.06255973]]
pred_robot: [[  40.52824338]
 [ 401.68676665]
 [   1.        ]]
Prediction: [[  40.52824338]
 [ 401.68676665]
 [   0.        ]]
[[  2.76053065]
 [ 13.62016905]]
pred_robot: [[ 114.53432765]
 [  32.76619194]
 [   1.        ]]
Prediction: [[ 114.53432765]
 [  32.76619194]
 [   0.        ]]
[[ 5.12037919]
 [ 9.92000587]]
pred_robot: [[ 178.25023809]
 [ 132.53184183]
 [   1.        ]]
Prediction: [[ 178.25023809]
 [ 132.53184183]
 [   0.        ]]
